In [4]:
from songs import *

# ОКИ...

In [5]:
import numpy as np
from numpy import random

In [6]:
MEMORY_TIMES = 64 # храним последние 8 тактов. 8 тактов сходится лучше, чем 4 :/
STARTER = 8

MEMORY_SIZE = MEMORY_TIMES * 13 

In [7]:
NAMES = []
for t in range(-MEMORY_TIMES, 0):
    for note_name in ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B", "C^"]:
        NAMES.append(note_name + str(t))

In [8]:
# если качество правила ниже этого порога, то правило нужно убить. 
# Выше 0 ставить бессмысленно, и похоже, что 0 оптимален
BAD_RULE = 0  

class Func:
    def __init__(self):
        self.operations = []
        self.operands = np.zeros((0, MEMORY_SIZE))
        self.cnt_nonzero = []
        
        self.final_decision_operation = -1  # индекс последнего задействованного правила
        self.rewards = []                   # качество каждого правила
    
    # последовательное применение правил
    def apply(self, x):
        ans = 0
        self.final_decision_operation = -1
        
        vals = self.operands.dot(x) == self.cnt_nonzero
        
        for index, oper, val in zip(range(len(self.operations)), self.operations, vals):
            if oper == "conj" and ans and val:
                ans = 0
                self.final_decision_operation = index
            if oper == "disj" and not ans and val:
                ans = 1
                self.final_decision_operation = index
        return ans
    
    def reward(self):  # награждаем
        if self.final_decision_operation != -1:
            self.rewards[self.final_decision_operation] += 1
    def hate(self):    # ненавидим
        if self.final_decision_operation != -1:
            self.rewards[self.final_decision_operation] -= 1
            
            if self.rewards[self.final_decision_operation] < BAD_RULE:
                del self.operations[self.final_decision_operation]
                del self.rewards[self.final_decision_operation]
                del self.cnt_nonzero[self.final_decision_operation]
                self.operands = np.delete(self.operands, self.final_decision_operation, 0)
                return True
        return False
    
    # добавляем новое правило
    def add(self, operation, included_in_operand, where_to_put):
        self.operations.insert(where_to_put, operation)
        self.rewards.insert(where_to_put, 1)
        
        mask = np.zeros((MEMORY_SIZE))
        for ones in included_in_operand:
            mask[ones] = 1
        self.operands = np.vstack([self.operands[:where_to_put], mask, self.operands[where_to_put:]])
                                  
        self.cnt_nonzero.insert(where_to_put, np.count_nonzero(mask))
    
    # вывод правила на экран
    def print(self):
        print("0", end="")
        for oper, new_operand in zip(self.operations, self.operands):
            if oper == "conj":
                print(" ∧ not", end="")
            if oper == "disj":
                print(" ∨ ", end="")
            for i in range(MEMORY_SIZE):
                if new_operand[i]:
                    print(NAMES[i], end="")
        print("")

In [9]:
class Player:
    def __init__(self):
        self.play_note = [Func() for i in range(13)]
        self.history = [set() for i in range(13)]
        self.Random = np.random.RandomState(seed=179)
        
    def play(self, memory):
        return np.array([self.play_note[i].apply(memory) for i in range(13)], dtype=int)
    
    def check(self, note, impulses):
        return not frozenset(impulses.flatten()) in self.history[note]
    
    def turn_on(self, note, memory):
        # берём множество сигналов и перемешиваем
        impulses = memory.nonzero()[0]
        self.Random.shuffle(impulses)
        
        # ищем новую уникальную эл. конъюнкцию
        cut = 1
        while cut <= len(impulses) and not self.check(note, impulses[:cut]):
            cut += 1
        
        # не нашли - беда. Тут я схожу с ума и СТИРАЮ всю функцию, ого.
        if cut > len(impulses):
            print("data error: contradiction found")
            cut = 3
            self.play_note[note] = Func()
        
        # добавляем новое правило
        self.play_note[note].add("disj", impulses[:cut], self.play_note[note].final_decision_operation + 1)
        # сохраняем конъюнкцию в списке
        self.history[note].add(frozenset(impulses[:cut].flatten()))
        #self.play_note[note].print()
    
    # аналогичная функция
    def turn_off(self, note, memory):
        impulses = memory.nonzero()[0]
        self.Random.shuffle(impulses)
        
        cut = 1
        while cut <= len(impulses) and not self.check(note, impulses[:cut]):
            cut += 1
            
        if cut > len(impulses):
            print("data error: contradiction found")
            cut = 3
            self.play_note[note] = Func()
        
        self.play_note[note].add("conj", impulses[:cut], self.play_note[note].final_decision_operation + 1)
        self.history[note].add(frozenset(impulses[:cut].flatten()))
        #self.play_note[note].print()
        
    def learn_to_play(self, song_to_learn, verbose=False):
        # инициализация
        memory = np.zeros((13 * MEMORY_TIMES), dtype=int)
        memory[-STARTER*13:] = song_to_learn.notes[:STARTER].flatten()

        t = STARTER
        errors = 0

        while t < len(song_to_learn.notes):    
            output = self.play(memory)  # то, что играется правилом сейчас
            
            if verbose:
                print(output)

            for note in range(13):    
                if output[note] != song_to_learn.notes[t][note]:  # если ошиблись
                    if not self.play_note[note].hate():           # возможно, ненависть уже удалила нехорошее правило
                        if song_to_learn.notes[t][note] == 1:     # иначе нужно "включить" или "выключить" клавишу
                            errors += 1
                            self.turn_on(note, memory)
                            if verbose:
                                print("note " + str(note) + " should be pushed!")
                        if song_to_learn.notes[t][note] == 0:
                            errors += 1
                            self.turn_off(note, memory)
                            if verbose:                            
                                print("note " + str(note) + " should not be pushed!")
                    elif verbose:                            
                                print("note " + str(note) + " caused hating!")
                else:
                    self.play_note[note].reward()                 # если всё хорошо, награждаем крутое правило
            
            # переходим к следующему шагу
            memory = np.append(memory[13:], song_to_learn.notes[t])
            t += 1
        
        if verbose:
            print("Num of Errors: ", errors)
        return errors
    
    # импровизация
    def improvise(self, starter):
        memory = np.zeros((13 * MEMORY_TIMES), dtype=int)
        memory[-STARTER*13:] = starter.notes[:STARTER].flatten()

        result = MySong(starter.notes[:STARTER])
        
        t = STARTER
        while t < len(kuznechik.notes):    
            output = player.play(memory)
            result.add(output)

            memory = np.append(memory[13:], output)
            t += 1
        result.finish()
        
        return result

## Обучение

Создаём нового игрока и грузим пока кузнечика

In [10]:
player = Player()

In [11]:
kuznechik = Song('basic midi/track (1).mid')

Учим игрока играть кузнечика без ошибок. seed задан, это занимает 19 итераций. С эвристикой позиции нового правила - 14. А с эвристикой ненависти - 12.

In [11]:
errors = -1
while errors != 0:
    errors = player.learn_to_play(kuznechik)
    print(errors)

note 8 caused hating!
note 9 caused hating!
note 4 caused hating!
note 4 caused hating!
note 4 caused hating!
note 4 caused hating!
note 12 caused hating!
note 11 caused hating!
note 11 caused hating!
note 9 caused hating!
note 12 caused hating!
note 9 caused hating!
note 9 caused hating!
note 9 caused hating!
66
note 8 caused hating!
note 4 caused hating!
note 12 caused hating!
note 11 caused hating!
note 12 caused hating!
note 11 caused hating!
note 11 caused hating!
note 4 caused hating!
note 4 caused hating!
note 4 caused hating!
note 12 caused hating!
note 4 caused hating!
note 8 caused hating!
note 11 caused hating!
note 11 caused hating!
note 12 caused hating!
note 12 caused hating!
note 9 caused hating!
note 8 caused hating!
note 8 caused hating!
note 11 caused hating!
note 11 caused hating!
74
note 9 caused hating!
note 8 caused hating!
note 4 caused hating!
note 11 caused hating!
note 12 caused hating!
note 12 caused hating!
note 12 caused hating!
note 8 caused hating!
note 1

Проверим, что всё окей. Подадим на вход начало кузнечика, а дальше пусть играет сам (память заполняется тем, что игрок сам нажимает на рояле). Поскольку так заданы правила, кузнечик будет воспроизведён точно

In [ ]:
player.improvise(kuznechik).play()

Вот как выглядит его булево правило (для ноты ми):

In [12]:
player.play_note[4].print()

0 ∨ E-32 ∧ notB-16 ∧ notG#-24 ∨ E-28 ∨ G#-8 ∧ notA-20 ∧ notE-40 ∧ notG#-52 ∧ notG#-46 ∧ notE-14 ∧ notG#-2 ∨ A-14 ∧ notG#-62 ∧ notA-10G#-2 ∨ E-12G#-2 ∧ notE-12E-8G#-2 ∧ notA-38 ∧ notA-50E-42 ∧ notG#-36A-14 ∧ notG#-46C^-22 ∧ notA-30 ∨ G#-14A-6 ∧ notG#-58 ∧ notA-30G#-28 ∨ E-32A-10 ∧ notG#-16 ∧ notA-4


Окей, чтобы услышать что-то новое, нужно другое начало. Возьмём его из другой песенки

In [13]:
simple_song = Song('basic midi/track (2).mid')
result = player.improvise(simple_song)

Первый шедевр:

In [14]:
result.play()

Игрок пока использует только несколько нот. Просто в кузнечике есть не все 13 нот, так что мы можем его посмещать, чтобы получить новую информацию. Попробуем сместить кузнечика на одну ноту.

In [77]:
def addAllTransposedVersions(Songs, song):
    while song.transpose(1):
        pass

    Songs.append(copy.deepcopy(song))
    while song.transpose(-1):
        Songs.append(copy.deepcopy(song))

Возьмём транспонированные версии первых 6 песенок и попробуем на них обучиться.

In [78]:
Songs = []
addAllTransposedVersions(Songs, Song('basic midi/track (1).mid'))
addAllTransposedVersions(Songs, Song('basic midi/track (2).mid'))
addAllTransposedVersions(Songs, Song('basic midi/track (3).mid'))
addAllTransposedVersions(Songs, Song('basic midi/track (4).mid'))
addAllTransposedVersions(Songs, Song('basic midi/track (5).mid'))
addAllTransposedVersions(Songs, Song('basic midi/track (6).mid'))

In [79]:
i = 0
learned_songs = 0
epoch = 0
epoch_errors = 0

while learned_songs != len(Songs):
    print("Song " + str(i) + " started:")

    errors = player.learn_to_play(Songs[i])
    print(errors)
    epoch_errors += errors
    
    if errors == 0:
        learned_songs += 1
    else:
        learned_songs = 0
    
    #while errors != 0:
    #    errors = player.learn_to_play(Songs[i])
    #    print(errors)
    
    i = (i + 1) % len(Songs)
    if i == 0:
        epoch += 1
        print("Epoch errors: " + str(epoch_errors / len(Songs)))
        print("NEW EPOCH " + str(epoch))
        epoch_errors = 0

Song 0 started:
66
Song 1 started:
75
Song 2 started:
73
Song 3 started:
64
Song 4 started:
65
Song 5 started:
52
Song 6 started:
59
Song 7 started:
62
Song 8 started:
81
Song 9 started:
56
Song 10 started:
72
Song 11 started:
73
Song 12 started:
70
Song 13 started:
55
Song 14 started:
73
Song 15 started:
56
Song 16 started:
57
Song 17 started:
65
Song 18 started:
60
Song 19 started:
65
Song 20 started:
73
Song 21 started:
55
Song 22 started:
54
Song 23 started:
63
Song 24 started:
68
Song 25 started:
49
Song 26 started:
66
Song 27 started:
67
Song 28 started:
59
Song 29 started:
58
Song 30 started:
52
Song 31 started:
56
Song 32 started:
53
Song 33 started:
58
Song 34 started:
63
Song 35 started:
57
Song 36 started:
66
Song 37 started:
54
Song 38 started:
55
Song 39 started:
59
Song 40 started:
74
Song 41 started:
57
Song 42 started:
75
Song 43 started:
80
Song 44 started:
90
Song 45 started:
76
Song 46 started:
59
Song 47 started:
66
Song 48 started:
71
Song 49 started:
49
Song 50 st

26
Song 51 started:
70
Song 52 started:
data error: contradiction found
55
Epoch errors: 51.16981132075472
NEW EPOCH 6
Song 0 started:
61
Song 1 started:
63
Song 2 started:
59
Song 3 started:
70
Song 4 started:
61
Song 5 started:
54
Song 6 started:
data error: contradiction found
59
Song 7 started:
51
Song 8 started:
59
Song 9 started:
47
Song 10 started:
57
Song 11 started:
45
Song 12 started:
59
Song 13 started:
40
Song 14 started:
48
Song 15 started:
43
Song 16 started:
50
Song 17 started:
35
Song 18 started:
35
Song 19 started:
61
Song 20 started:
68
Song 21 started:
56
Song 22 started:
data error: contradiction found
data error: contradiction found
31
Song 23 started:
data error: contradiction found
32
Song 24 started:
data error: contradiction found
33
Song 25 started:
data error: contradiction found
data error: contradiction found
36
Song 26 started:
44
Song 27 started:
52
Song 28 started:
33
Song 29 started:
data error: contradiction found
data error: contradiction found
42
Son

54
Song 1 started:
45
Song 2 started:
55
Song 3 started:
59
Song 4 started:
58
Song 5 started:
data error: contradiction found
42
Song 6 started:
data error: contradiction found
48
Song 7 started:
49
Song 8 started:
53
Song 9 started:
data error: contradiction found
40
Song 10 started:
data error: contradiction found
45
Song 11 started:
30
Song 12 started:
39
Song 13 started:
38
Song 14 started:
data error: contradiction found
35
Song 15 started:
data error: contradiction found
33
Song 16 started:
data error: contradiction found
41
Song 17 started:
22
Song 18 started:
28
Song 19 started:
data error: contradiction found
56
Song 20 started:
66
Song 21 started:
43
Song 22 started:
data error: contradiction found
data error: contradiction found
data error: contradiction found
34
Song 23 started:
data error: contradiction found
data error: contradiction found
37
Song 24 started:
data error: contradiction found
data error: contradiction found
29
Song 25 started:
data error: contradiction fou

36
Song 17 started:
18
Song 18 started:
23
Song 19 started:
52
Song 20 started:
data error: contradiction found
data error: contradiction found
62
Song 21 started:
43
Song 22 started:
data error: contradiction found
data error: contradiction found
data error: contradiction found
24
Song 23 started:
data error: contradiction found
data error: contradiction found
29
Song 24 started:
data error: contradiction found
data error: contradiction found
29
Song 25 started:
data error: contradiction found
29
Song 26 started:
44
Song 27 started:
data error: contradiction found
40
Song 28 started:
data error: contradiction found
28
Song 29 started:
data error: contradiction found
data error: contradiction found
data error: contradiction found
38
Song 30 started:
data error: contradiction found
39
Song 31 started:
data error: contradiction found
46
Song 32 started:
data error: contradiction found
51
Song 33 started:
47
Song 34 started:
data error: contradiction found
47
Song 35 started:
data error: 

35
Song 15 started:
data error: contradiction found
39
Song 16 started:
data error: contradiction found
42
Song 17 started:
21
Song 18 started:
22
Song 19 started:
data error: contradiction found
data error: contradiction found
49
Song 20 started:
data error: contradiction found
58
Song 21 started:
36
Song 22 started:
data error: contradiction found
data error: contradiction found
data error: contradiction found
31
Song 23 started:
data error: contradiction found
data error: contradiction found
32
Song 24 started:
data error: contradiction found
data error: contradiction found
26
Song 25 started:
data error: contradiction found
data error: contradiction found
30
Song 26 started:
42
Song 27 started:
data error: contradiction found
35
Song 28 started:
data error: contradiction found
37
Song 29 started:
data error: contradiction found
data error: contradiction found
data error: contradiction found
35
Song 30 started:
data error: contradiction found
31
Song 31 started:
data error: contradi

...

25
Song 17 started:
13
Song 18 started:
16
Song 19 started:
data error: contradiction found
data error: contradiction found
25
Song 20 started:
data error: contradiction found
data error: contradiction found
data error: contradiction found
47
Song 21 started:
data error: contradiction found
data error: contradiction found
27
Song 22 started:
data error: contradiction found
data error: contradiction found
data error: contradiction found
data error: contradiction found
17
Song 23 started:
data error: contradiction found
data error: contradiction found
data error: contradiction found
19
Song 24 started:
data error: contradiction found
data error: contradiction found
data error: contradiction found
23
Song 25 started:
data error: contradiction found
data error: contradiction found
data error: contradiction found
25
Song 26 started:
27
Song 27 started:
data error: contradiction found
data error: contradiction found
28
Song 28 started:
data error: contradiction found
data error: contradictio

27
Song 48 started:
23
Song 49 started:
data error: contradiction found
data error: contradiction found
13
Song 50 started:
data error: contradiction found
data error: contradiction found
data error: contradiction found
data error: contradiction found
data error: contradiction found
15
Song 51 started:
data error: contradiction found
data error: contradiction found
42
Song 52 started:
data error: contradiction found
24
Epoch errors: 27.962264150943398
NEW EPOCH 60
Song 0 started:
data error: contradiction found
data error: contradiction found
26
Song 1 started:
data error: contradiction found
data error: contradiction found
data error: contradiction found
25
Song 2 started:
data error: contradiction found
data error: contradiction found
data error: contradiction found
35
Song 3 started:
data error: contradiction found
data error: contradiction found
data error: contradiction found
48
Song 4 started:
data error: contradiction found
data error: contradiction found
data error: contradicti

19
Song 29 started:
data error: contradiction found
data error: contradiction found
data error: contradiction found
data error: contradiction found
23
Song 30 started:
data error: contradiction found
data error: contradiction found
data error: contradiction found
data error: contradiction found
data error: contradiction found
25
Song 31 started:
data error: contradiction found
30
Song 32 started:
data error: contradiction found
data error: contradiction found
data error: contradiction found
37
Song 33 started:
data error: contradiction found
data error: contradiction found
30
Song 34 started:
data error: contradiction found
data error: contradiction found
31
Song 35 started:
data error: contradiction found
data error: contradiction found
data error: contradiction found
data error: contradiction found
28
Song 36 started:
data error: contradiction found
30
Song 37 started:
data error: contradiction found
data error: contradiction found
29
Song 38 started:
data error: contradiction found


35
Song 7 started:
data error: contradiction found
28
Song 8 started:
data error: contradiction found
32
Song 9 started:
data error: contradiction found
data error: contradiction found
data error: contradiction found
27
Song 10 started:
data error: contradiction found
data error: contradiction found
data error: contradiction found
28
Song 11 started:
data error: contradiction found
data error: contradiction found
data error: contradiction found
18
Song 12 started:
data error: contradiction found
data error: contradiction found
data error: contradiction found
27
Song 13 started:
data error: contradiction found
data error: contradiction found
data error: contradiction found
20
Song 14 started:
data error: contradiction found
data error: contradiction found
data error: contradiction found
31
Song 15 started:
data error: contradiction found
data error: contradiction found
23
Song 16 started:
data error: contradiction found
data error: contradiction found
28
Song 17 started:
11
Song 18 star

KeyboardInterrupt: 

In [52]:
simple_song = Song('test/whomadethis.mid')
result = player.improvise(simple_song)
result.play()

In [37]:
result.save_file("dog waltz.mid")